In [1]:
import pandas as pd
import numpy as np

In [2]:
listings = pd.read_csv('../originalData/listings.csv')
key_columns = [
    "id", "price", "room_type", "property_type", "accommodates",
    "bedrooms", "beds", "bathrooms", "amenities",
    "neighbourhood_cleansed", "latitude", "longitude",
    "host_is_superhost", "host_listings_count", "instant_bookable",
    "review_scores_rating",
    "availability_30", "availability_60", "availability_90", "availability_365"
    ]

listings = pd.read_csv(
    '../originalData/listings.csv',
    usecols=lambda col: col in key_columns
)

print(listings.head())

         id host_is_superhost  host_listings_count neighbourhood_cleansed  \
0  12147973                 f                    1             Roslindale   
1   3075044                 f                    1             Roslindale   
2      6976                 t                    1             Roslindale   
3   1436513                 f                    1             Roslindale   
4   7651065                 t                    1             Roslindale   

    latitude  longitude property_type        room_type  accommodates  \
0  42.282619 -71.133068         House  Entire home/apt             4   
1  42.286241 -71.134374     Apartment     Private room             2   
2  42.292438 -71.135765     Apartment     Private room             2   
3  42.281106 -71.121021         House     Private room             4   
4  42.284512 -71.136258         House     Private room             2   

   bathrooms  bedrooms  beds  \
0        1.5       2.0   3.0   
1        1.0       1.0   1.0   
2       

In [3]:
#Make each column name easy to read
listings.columns = (listings.columns.str.strip().str.lower().str.replace(r"\s+", "_"))

In [4]:
#Clean up our price column, convert it to a float
listings["price"] = (listings["price"]
                     .astype(str).
                     str.replace(",", "").
                     str.replace("$", "").
                     replace("", np.nan)
                     .astype(float))
print(listings["price"].info())

#keep only the prices that make sense
listings = listings.dropna(subset=["price"])
listings = listings[listings["price"] > 0]

#remidner for tmrw that after were done cleanign we create this as a cleanedCSV then its time to combine the three cleaned csvs into a single big where our operatiosn will be preformed yayayaya

<class 'pandas.core.series.Series'>
RangeIndex: 3585 entries, 0 to 3584
Series name: price
Non-Null Count  Dtype  
--------------  -----  
3585 non-null   float64
dtypes: float64(1)
memory usage: 28.1 KB
None


In [5]:
#Lets count how amenities are listed for each airbnb
listings["amenities_count"] = (
    listings["amenities"]
        .astype(str)
        .str.replace(r"^\[|\]", "", regex=True)
        .apply(lambda s: len([x for x in s.split(",") if x.strip()]) if s not in ["nan", "None", "none"] else 0)
)
print(listings["amenities_count"].info())
print(listings[["amenities", "amenities_count"]].head())

<class 'pandas.core.series.Series'>
RangeIndex: 3585 entries, 0 to 3584
Series name: amenities_count
Non-Null Count  Dtype
--------------  -----
3585 non-null   int64
dtypes: int64(1)
memory usage: 28.1 KB
None
                                           amenities  amenities_count
0  {TV,"Wireless Internet",Kitchen,"Free Parking ...               15
1  {TV,Internet,"Wireless Internet","Air Conditio...               21
2  {TV,"Cable TV","Wireless Internet","Air Condit...               18
3  {TV,Internet,"Wireless Internet","Air Conditio...               23
4  {Internet,"Wireless Internet","Air Conditionin...               14


In [6]:
#the is_superhost and instant_bookable columns both arent booleans, lets make them booleans
listings["host_is_superhost"] = listings["host_is_superhost"].map({
    't' : True,
    'f' : False
})
listings["instant_bookable"] = listings["instant_bookable"].map({
    't' : True,
    'f' : False
})

print(listings[["host_is_superhost", "instant_bookable"]].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   host_is_superhost  3585 non-null   bool 
 1   instant_bookable   3585 non-null   bool 
dtypes: bool(2)
memory usage: 7.1 KB
None


In [7]:
#before we get to the end lets make sure all of our columns that contain numeric values are actually all numeric values
num_cols = [
    "accommodates", "bedrooms", "beds", "latitude", "longitude",
    "host_listings_count", "review_scores_rating",
    "availability_30", "availability_60", "availability_90", "availability_365"
]

for c in num_cols:
    if c in listings.columns:
        listings[c] = pd.to_numeric(listings[c], errors="coerce")

In [8]:
#remove any duplicate listing ids
listings = listings.drop_duplicates(subset=["id"], keep="last")

In [9]:
#lets sort our airbnb prices into L / M / H labels using mean bin smoothing!!!!

# Sort prices and split into 3 equal bins
sorted_price = listings['price'].sort_values().reset_index(drop=True)
bins = np.array_split(sorted_price.to_numpy(), 3)

# Compute mean price per bin
bin_means = [float(b.mean()) for b in bins]

# Define bin edges
bin_edges = [-np.inf, bins[0].max(), bins[1].max(), np.inf]

# Assign categorical labels
labels = ['Budget', 'Average', 'Expensive']
listings['price_category'] = pd.cut(
    x=listings['price'],
    bins=bin_edges,
    labels=labels,
    include_lowest=True
)

In [10]:
listings = listings.dropna()

listings.to_csv("../cleanedData/cleanedListings.csv", index=False)